In [ ]:
from collections import Counter
import cv2
import math
import numpy as np
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import random
from random import randint
import re
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH = "/content/drive/My Drive/pictures"

In [ ]:
def files_in_folder(folder_path):
  '''
  Returns a list of strings where each entry is a file in the folder_path.
  
  Parameters
  ----------
  
  folder_path : str
     A string to folder for which the file listing is returned.
     
  '''
  files_A = !ls "{folder_path}"
  # The files when listed from Google Drive have a particular format. They are
  # grouped in sets of 4 and have spaces and tabs as delimiters.
  
  # Split the string listing sets of 4 files by tab and space and remove any 
  # empty splits.
  files_B = [list(filter(None, re.split('\t|\s', files))) for files in files_A]
  
  # Concatenate all splits into a single sorted list
  files_C = []
  for element in files_B:
    files_C = files_C + element
  files_C.sort()
  
  return files_C

files = files_in_folder(PATH)
print(files)

['plate_AA58.png', 'plate_AA63.png', 'plate_AA67.png', 'plate_AB02.png', 'plate_AC04.png', 'plate_AC55.png', 'plate_AC65.png', 'plate_AC96.png', 'plate_AD07.png', 'plate_AD76.png', 'plate_AE37.png', 'plate_AE44.png', 'plate_AE58.png', 'plate_AE84.png', 'plate_AF09.png', 'plate_AF73.png', 'plate_AG00.png', 'plate_AG09.png', 'plate_AG92.png', 'plate_AH58.png', 'plate_AI18.png', 'plate_AJ06.png', 'plate_AJ12.png', 'plate_AK29.png', 'plate_AK83.png', 'plate_AL07.png', 'plate_AL53.png', 'plate_AL79.png', 'plate_AM79.png', 'plate_AN74.png', 'plate_AN83.png', 'plate_AO60.png', 'plate_AO65.png', 'plate_AP83.png', 'plate_AQ34.png', 'plate_AQ35.png', 'plate_AQ53.png', 'plate_AS24.png', 'plate_AS42.png', 'plate_AT02.png', 'plate_AT03.png', 'plate_AT31.png', 'plate_AU29.png', 'plate_AU96.png', 'plate_AX22.png', 'plate_AX55.png', 'plate_AX68.png', 'plate_AX88.png', 'plate_AY94.png', 'plate_AZ40.png', 'plate_BA46.png', 'plate_BB09.png', 'plate_BB72.png', 'plate_BD38.png', 'plate_BD71.png', 'plate_BD

In [ ]:
test_plates = np.array([np.array(Image.open(f'{PATH}/{file}'))
                    for file in files[:]])

print(test_plates.shape)
print(len(files))

(991, 298, 600, 3)
991


In [ ]:
# We will use this custom filter function in addition to the standard 
# augmentation techniques. You will want to modify this function to fit your 
# needs.

def my_filter(image):
  '''
  @brief filter that introduces rescaling artifacts and changes the color 
         channel order to match plt.imshow() format
  @param image: input image to be modified
  @retval image with 
  '''
  resize_factor = 0.1
  initial_shape = image.shape
  #Need to reverse the order of width and height because PIL format is reversed 
  #from cv2 format.
  initial_shape = tuple(reversed(initial_shape[0:2])) + (initial_shape[2],)

  #resize height and width
  small_shape = tuple((np.array(initial_shape[0:2]) * resize_factor).astype(int))

  #scale down and scale back up to introduce rescaling artifacts
  res = cv2.resize(image, dsize=small_shape, interpolation=cv2.INTER_LINEAR)
  res = cv2.resize(res, dsize=initial_shape[0:2], interpolation=cv2.INTER_NEAREST)

  #rearrange the color channels so imshow displays the appropriate colors
  res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
  return res

In [ ]:
%tensorflow_version 1.14.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

In [ ]:
#Create a data generating object - check out 
datagen = ImageDataGenerator(fill_mode='constant',
                             preprocessing_function=my_filter,
                             zoom_range=0.1,
                             width_shift_range=0.1, height_shift_range=0.1,
                             brightness_range=[0.2,1.3],
                             rotation_range=5)

aug_iter = datagen.flow(test_plates, batch_size=1)

In [ ]:
for i in range(991):
  image = next(aug_iter)[0].astype('uint8')
  cv2.imwrite(os.path.join(PATH, "plate_{}.png".format(i)), image)